# Koch Snowflake

## Background

The Koch snowflake is a mathematical curve representing the Koch curve. it can be built by starting with an equilateral triangle, removing the inner third of each side, and building a seperate triangle where the side was removed. this process can be repeated indefinitely, creating a surface with a finite area, but an infinite perimeter.

The image below shows an example of a Koch snowflake.

<img src = "images/Snowflake.png" style = "width:368px; height:330px"/>

## Software Version

The code below draws the Koch Snowflake using Python's matplotlib library.

The first cell imports the required libraries.

In [ ]:
import random as rd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from math import *
from ipywidgets import *

The second cell defines some key values that will be used in the code.
sin60 is used to define the height of the initial triangle, followed by the heights of the added triangles at every iteration.
xdata and ydata represent the starting points for X and Y

In [ ]:
sin60 = sqrt(3)/2
count = int(1)
xbase = [0, 0.5, 1, 0]
ybase = [0, sin60, 0, 0]

The third cell defines the removal of the middle third of each line, as well as forming the basis for the next iteration.

In [ ]:
def linebreak(pos, xdata, ydata):

    x0 = xbase[pos]
    x1 = xbase[pos+1]
    y0 = ybase[pos]
    y1 = ybase[pos+1]
    xvec = x1 - x0
    yvec = y1 - y0

    xbase.insert(pos+1, x0 + xvec/3)
    ybase.insert(pos+1, y0 + yvec/3)

    xbase.insert(pos+2, x0 + xvec/2 - yvec*sin60/3)
    ybase.insert(pos+2, y0 + xvec*sin60/3 + yvec/2)

    xbase.insert(pos+3, x0 + 2*xvec/3)
    ybase.insert(pos+3, y0 + 2*yvec/3)

The fourth cell defines the addition of the new equilateral triangle.

In [ ]:
def koch_iteration(xbase, ybase):

    pos = 0
    while pos < len(xbase) - 1:
        linebreak(pos, xbase, ybase)
        pos += 4

The fifth cell then provides the final Koch snowflake, whilst allowing the user to define the amount of iterations they want it to go through.

In [ ]:
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
ax.axes.set_xlim(-1.5,1.5)
ax.axes.set_ylim(-1.5, 2.0)
fig.tight_layout()
plt.show

for i in range(1,4):
    koch_iteration(xbase, ybase)
    ax.plot(xbase, ybase)
    count +=1
    plt.show


The final cell will be replaced by a hardware integrated method, as it allows displays the area of the final snowflake's area.

In [ ]:
k = int(count+1)
totalarea = 0
for i in range (1,k): 
    fours = 4**(i-1)
    nines = 9**i
    pre = (2**2)*(sqrt(3))/4
    area = k* (pre*(3*fours/nines))
    totalarea +=area

print (totalarea)

## Hardware Version

To simplify some of the calculations, an IP core was designed that would create the loop for k, instead of calculating it in the software. the loop adder was based off of the Simple Adder that was shown in Lab 3, though with the output serving as one of the inputs instead. so long as Input A remains constant, this adder will increase linearly.

<img src = "images/Sysgen.png" style = "width:825px; height:464px"/>

This loop will be used on the count integer from the software version.

The following cell imports the Overlay class from the Pynq library. Following this, it creates a new overlay and parse the design to access the contents of said overlay. the required libraries are also imported.

In [ ]:
from pynq import Overlay
ol = Overlay("design_loop_wrapper.bit")
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patch
from math import *
from ipywidgets import *

the proceeding cell returns a report of the overlay which will show the IP core under the "IP Blocks" heading

In [ ]:
ol?

the next cell simplifies the core's name so that it can be easier to reference

In [ ]:
Loop = ol.looped_add_0

the next cells are relatively unchanged from the software implementation

In [ ]:
sin60 = sqrt(3)/2
xbase = [0, 0.5, 1, 0]
ybase = [0, sin60, 0, 0]

In [ ]:
def linebreak(pos, xdata, ydata):

    x0 = xbase[pos]
    x1 = xbase[pos+1]
    y0 = ybase[pos]
    y1 = ybase[pos+1]
    xvec = x1 - x0
    yvec = y1 - y0

    xbase.insert(pos+1, x0 + xvec/3)
    ybase.insert(pos+1, y0 + yvec/3)

    xbase.insert(pos+2, x0 + xvec/2 - yvec*sin60/3)
    ybase.insert(pos+2, y0 + xvec*sin60/3 + yvec/2)

    xbase.insert(pos+3, x0 + 2*xvec/3)
    ybase.insert(pos+3, y0 + 2*yvec/3)

The following cell defines a function to call on the loop.

In [ ]:
def Looped (count):
    Loop.write (0x00, int(count))
    K = Loop.read(0x08)
    return K

The following cells remain unchanged from the previous version.

In [ ]:
def koch_iteration(xbase, ybase):

    pos = 0
    while pos < len(xbase) - 1:
        linebreak(pos, xbase, ybase)
        pos += 4

In [ ]:
fig, ax = plt.subplots()
ax.axis('equal')
ax.axis('off')
ax.axes.set_xlim(-1.5,1.5)
ax.axes.set_ylim(-1.5, 2.0)
fig.tight_layout()
plt.show

for i in range(1,4):
    koch_iteration(xbase, ybase)
    ax.plot(xbase, ybase)
    Looped(1)
    plt.show

The cell below provides a calculation of the total area of this triangle, using the loop function to increase the count at each step.

In [ ]:
k = Looped(4)+1
totalarea = 0
for i in range (1,k): 
    fours = 4**(i-1)
    nines = 9**i
    pre = (2**2)*(sqrt(3))/4
    area = k* (pre*(3*fours/nines))
    totalarea += area

print (totalarea)

The difference in value between the software and hardware implementations is caused by the hardware counting one iteration less than the software, this can be rectified by adding 1 to the looped value as shown in the above code cell.